# Machine Learnign vs. March Madness

## Overview

Sports gambling is one of the fastest growing industries in the country, with states continuing to pass betting-friendly legislation and companies like DraftKings, FanDuel and BetMGM experiencing increased, year-over-year revenue. One of the premier sports betting events of the year is the NCAA college basketball postseason tournament, commonly known as March Madness.

This project posits that we run a sports gambling company. We offer our customers advisory services during the busiest time of the sports year on events with the greatest amount of betting action.

We target college basketball because of our domain knowledge and the talent disparity within the sport. In professional sports, the talent gap between the best and worst teams is very small. It's unusual to see NBA spreads greater than 10 points. In college basketball, this happens regularly. Teams are 25-point underdogs on a given night and that creates betting value. There also tends to be more regular competition in college basketball. With only around 30 games in a season, teams don't have the luxury to take nights off like we see in an 82-game NBA season. And with constant effort comes more predictable outcomes.

## Business Objective

To maximize returns, we run a series of machine learning algorithms to model predictions for single games in the NCAA tournament. We pay particular focus to accurately predicting underdog team wins, as doing so yields higher payouts. Accuracy, and more specifically predictive accuracy, is paramount in selecting our models, as we strive to minimize risk for our customers.

## Data

This project uses datasets from Kaggle's March Machine Learning Mania 2021 competition.

## Methogology

We set the win/loss outcome for the favored team as the binary target variable, with 1 equaling a win for the favored team and 0 equaling a win for the underdog. Rankings are assigned using the reputable KenPom ratings.

We then use an iterative approach to build 6 predictive, classification models: Logistic Regression, K-Nearest Neighbors, Decision Tree, Random Forest, Bagging classifier and XGBoost. We utilize hyperparameter tuning, cross-validation and scoring to select the highest performing, predictive models. This approach is applied to regular season, postseason and cumulative postseason data.

# Data Cleaning

The files we downloaded from kaggle include in game stats for all seasons from 2003 up to 2020. Our predcitions are going to be based off the season average stats of the two relevant teams in each game. In order to get this we have to calculate the averages from all the games they played up to that point in the season, and make a new dataframe for the specific game with averages as input.

We then need to make a target variable. The difficulty with this dataset is, we would like to predict whether the favorite team (based on Ken Pom rankings) won the game for betting purposes, and this dataset does not include a team raking. We had to use another dataset from kaggle which includes the team rankings, check if the winning team in the game was the favorite, and then make a binary column of Favorite Win. 

Ken Pomeroy, the creator of KenPom, uses analytical measures to rank each team in college basketball. He is one of the most popular and well-known names in basketball analytics and has been at the forefront of the progression mainstream acceptance of advanced stats.

Once we have a target column, we can create a dataframe with full seasons worth of games with a target column, and train our model on it.

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# Importing a regular season dataframe, a postseason dataframe, a rankings dataframe, 
# and a dataframe with Ken Pom ranking from the rankings dataframe. 

regularseason = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
postseason = pd.read_csv('MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('MDataFiles_Stage1/MMasseyOrdinals.csv')
kenpom = rankings[(rankings['SystemName'] == 'POM') & (rankings['RankingDayNum'] >= 128)]

We want to have some more advanced stats to use for prediction. We create them here with a few calculations from the regular stats.

In [3]:
def add_rate_cols(df):
    # Calculates the number of possesions for each team in each game. The number of possessions
    # has a large effect on teams averages and counting stats. Knowing this tells us how
    # efficient a team is different factors of a game.
    df['possessions'] = .5 * (df['FGA'] + (.475 * df['FTA']) - df['OR'] + df['TO'])
    
    # Effective Field Goal Percentage (eFG) gives extra value to 3-point shots. Each 3-pointer made
    # is given the weight of 1.5 2-pointers made.
    df['eFG'] = (df['FGM'] + (.5 * df['FGM3'])) / df['FGA']
    
    # True Shooting Percentage (TS%) gives weight to the 3-point shot as well as free throws made.
    # This stat takes in account all types of scoring to determine a players shooting percentage.
    df['TS%'] = df['Score'] / (2 * (df['FGA'] + .475 * df['FTA']))
    
    # Free Throw Rate (FTr) is a simple ratio of Free Throws to Field Goals Attempted. It identifies
    # a team's abiilty to draw fouls and get to the free throw line where expected value is high.
    df['FTr'] = df['FTA'] / df['FGA']
    
    # Three Point Attempt Rate (3PAr) shows how often a player shoots 3-pointers compared to other
    # types of shots. It identifies players who attack the basket and draw fouls and players who are
    # likely to stand outside and shoot.
    df['3PAr'] = df['FGA3'] / (df['FGA'] + (.475*df['FTA']))
    
    # Offensive Rebound Percentage (OR%) is the number of offensive rebounds a team gets compared
    # to the total number of rebounds available while they were on offense.
    df['OR%'] = df['OR'] / (df['OR'] + df['Opp_DR'])
    
    # Defensive Rebound Percentage (DR%) is the number of defensive rebounds a team gets compared
    # to the total number of rebounds available while they were on defense.
    df['DR%'] = df['DR'] / (df['DR'] + df['Opp_OR'])
    
    # Rebound Percentage (REB%) is the percentage of rebounds from all available rebounds a team
    # gets in a game.
    df['REB%'] = (df['OR'] + df['DR']) / (df['OR'] + df['Opp_DR'] + df['Opp_OR'] + df['DR'])
    
    # Total Rebounds (TR) the number of rebounds a team gets in a game.
    df['TR'] = (df['OR'] + df['DR'])
    
    # Assist to Turnover Ratio (ATOr) shows how well a team takes care of the ball. The higher the
    # the number the better a team is at passing the ball and playing together.
    df['ATOr'] = df['Ast'] / df['TO']
    
    # Assist Percentage (Ast%) is the number of assists a team has compared to baskets scored. The
    # higher this percentage the better the team is at playing together and creating baskets for
    # one another.
    df['Ast%'] = df['Ast'] / df['FGM']
    
    # Steal Percentage (Stl%) is the rate at which a team gets steals during other teams possessions.
    df['Stl%'] = df['Stl'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])
    
    # Block Percentage (Blk%) is the rate at which a team blocks shots during other teams possessions.
    df['Blk%'] = df['Blk'] / (df['Opp_FGA'])
    
    # Turnover Percentage (TO_r) is the rate at which a team turnovers the ball over during their own
    # possessions.
    df['TO_r'] = df['TO'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    
    # Opponent Effective Field Goal Percentage (Opp_eFG) is a defensive measure that shows the Effective
    # Field Goal Percentage of a team's opponents.
    df['Opp_eFG'] = (df['Opp_FGM'] + (.5 * df['Opp_FGM3'])) / df['Opp_FGA']
    
    # Opponent True Shooting Percentage (Opp_TSpct) is a defensive measure that shows the True Shooting
    # Percentage of a team's opponents.
    df['Opp_TSpct'] = df['Opp_Score'] / (2 * (df['Opp_FGA'] + .475 * df['Opp_FTA']))
    
    # Opponent Free Throw Rate (Opp_FTr) is a defensive measure that shows how often a team fouls their
    # opponents and allows them to shoot free throws.
    df['Opp_FTr'] = df['Opp_FTA'] / df['Opp_FGA']
    
    # Opponent Three Point Rate (Opp_3Pr) is a defensive rate measure of how many 3-pointers an
    # opponent makes per basket scored.
    df['Opp_3Pr'] = df['Opp_FGA3'] / (df['Opp_FGA'] + (.475*df['Opp_FTA']))
    
    # Opponent Total Rebounds (Opp_TR) is the number of rebounds a team's opponent gets in a game.
    df['Opp_TR'] = (df['Opp_OR'] + df['Opp_DR'])
    
    # Opponent Assist to Turnover Ratio (Opp_ATOr) is a defensive rate measure of an opponents
    # assists per turnover. This is a good measure of defensive aggressiveness and the ability to make
    # opponents work hard for their shots.
    df['Opp_ATOr'] = df['Opp_Ast'] / df['Opp_TO']
    
    # Opponent Assist Percentage (Opp_Ast_%) is the percentage of opponent's baskets that are assisted.
    df['Opp_Ast_%'] = df['Opp_Ast'] / df['Opp_FGM']
    
    # Opponent Steal Percentage (Opp_Stl_%) is the percentage possesions that end with the opponent getting
    # a steal.
    df['Opp_Stl_%'] = df['Opp_Stl'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    
    # Opponent Block Percentage (Opp_Blk_%) is the percentage of opponent's possessions that end in a
    # block.
    df['Opp_Blk_%'] = df['Opp_Blk'] / (df['FGA'])
    
    # Opponent Turnover Rate (Opp_TO_r) is the percentage of opponent's possesions that end in a 
    # turnover.
    df['Opp_TO_r'] = df['Opp_TO'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])

    return df

This function will take all the in-game stats of a teams season, and put it into one dataframe. Then (if the paramter average is set to True), put that into a single row with the teams averages.

In [4]:
# This function is used to get the stats for a team in the regular season. The parameters are:
# the team id number, the season, the day number of the season, and whether you want to see
# each individual game or the season averages.

def team_regular_season(team_id,Season = all, DayNum = all, average = True):
    
    # create a dataframe that has all the wins/losses of a team during the season up to the given day number.
    teamwins = regularseason[(regularseason['WTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)] 
    teamlosses = regularseason[(regularseason['LTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)]
    
    # split the dataframes to only have the offensive statistics of the given team
    teamwinsoffense = teamwins.filter(like = 'W')    
    teamlossesoffense = teamlosses.filter(like = 'L')

    # split the dataframes to only have the defensive statistics of the given team
    teamwinsoffense.columns = teamwinsoffense.columns.str.lstrip('W')
    teamlossesoffense.columns = teamlossesoffense.columns.str.lstrip('L')

    # concat the two offensive dataframes
    teamoffense = pd.concat([teamwinsoffense, teamlossesoffense])

    # create oppposing team statistics from the given games
    teamwinsopponents = teamwins.filter(like = 'L')
    teamlossesopponents = teamlosses.filter(like = 'W')

    # rename the columns from the opponent statistics dataframes
    teamwinsopponents.columns = teamwinsopponents.columns.str.lstrip('L')
    teamlossesopponents.columns = teamlossesopponents.columns.str.lstrip('W')

    # concat the dataframes from opponent wins and losses
    teamopponents = pd.concat([teamwinsopponents, teamlossesopponents])
    
    # concat the team stats and the opponent stats to create a full picture of the team's season
    teamtotal = pd.concat([teamoffense, teamopponents.add_prefix('Opp_')], axis = 1)
    
    teamtotal.drop('Opp_TeamID', axis = 1, inplace = True)
    
    # return either the season averages or the game by game statistics
    if average == True:
        teamtotal = teamtotal.groupby('TeamID').mean().reset_index()
        add_rate_cols(teamtotal)
    
    if average == False:
        add_rate_cols(teamtotal)
        
    return teamtotal

In [5]:
# Here's an example of a team_regular_season output.
start = time.time()
marquette = team_regular_season(1266, 2020, 132, True)
end = time.time()
print('Execution time = %.6f seconds' % (end-start))
marquette

Execution time = 0.056832 seconds


Now we get the end of year rankings of a team for a specific year with this function.

In [7]:
def get_KenPom(team_id, Year):
    rank = kenpom[(kenpom['TeamID'] == team_id) & (kenpom['Season'] == Year)]
    rank.reset_index(inplace = True)
    return rank.OrdinalRank

get_KenPom(1181, 2020)

0    5
Name: OrdinalRank, dtype: int64

This function will take in a specified matchup and output a game with the two teams season stats and whether the favorite team one.

In [8]:
def single_game(team1_id, team2_id, Year = 2020, DayNum = 132):
    
    # from the Team IDs get a dataframe with the average seasonlong team statistics.
    team1 = team_regular_season(team1_id,Year,DayNum)
    team2 = team_regular_season(team2_id,Year,DayNum)
    
    # get the Ken Pom ranking for each team
    team1Pom = get_KenPom(team1_id, Year)
    team2Pom = get_KenPom(team2_id, Year)
    
    # due to the structure of the original dataframe, the first team listed is always the winner.
    # we check if the favorite won by comparing the rankings of the two teams.
    fav_win = pd.DataFrame(team1Pom.lt(team2Pom))
    
    # concatenate the dataframes and return the single game pre-game data.
    game = pd.concat([team1.add_prefix('W_'), team2.add_prefix('L_'), fav_win], axis = 1)
    game.rename(columns={game.columns[106]: 'fav_win'}, inplace = True)
    
    return game

This code shows how the stats from the function "single_game" update throughout the season. In this 
hypothetical example Duke plays North Carolina during the 2020 season on four separate occasions. On Day 0
there are no stats available so all the values are 'Nan'. On Day 2, only Duke has played a game, they have
stats available, however, North Carolina has not, so their values are all 'Nan'. On Day 50, both teams have
played and have available stats. On Day 100, both teams have continued deep into their seasons and have
updated statistics from the first 100 days of the year.

In [52]:
duke_unc = single_game(1181, 1314, 2020, 0)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [53]:
duke_unc = single_game(1181, 1314, 2020, 2)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,1181,68,23,64,8,24,14,23,11,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [55]:
duke_unc = single_game(1181, 1314, 2020, 25)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,1181,83.857143,29.714286,66.285714,7.0,21.0,17.428571,26.285714,16.428571,27.857143,...,0.465617,0.184896,0.40936,32.0,1.085714,0.527778,0.078417,0.03866,0.143524,True


In [10]:
duke_unc = single_game(1181, 1314, DayNum = 100)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,1181,82.583333,30.125,62.791667,6.958333,19.833333,15.375,21.958333,12.875,26.958333,...,0.517736,0.287568,0.358779,34.916667,1.122807,0.528053,0.082791,0.063984,0.146231,True




We can now write a function to create a full seasons worth of games using the single_game function to concatenate all of the single games of a season into one DataFrame. 

In [44]:
def create_season_df(df, season):
    
    # create a DataFrame for the season we are looking for
    season_df = df[(df['Season'] == season) & (df['DayNum'] >= 12)]
    season_df.reset_index(inplace = True)
    
    # create a list with the two teams, winning team always first, the season, and the day number of the game
    matchups = list(zip(season_df.WTeamID, season_df.LTeamID, season_df.Season, season_df.DayNum))
    
    season_games = []
    
    # iterate through the list we created and create each single game matchup throughout the season
    for i in range(len(matchups)):
        game = single_game(matchups[i][0], matchups[i][1], Year = matchups[i][2], DayNum = matchups[i][3])
        season_games.append(game)
    
    # from the season games list concatenate all the outputted DataFrames and insert the location
    # of the winner (home, away, neutral)
    df = pd.concat(season_games, axis = 0).dropna()
    df.reset_index(inplace = True, drop = True)
    df.insert(0,"w_loc", season_df.WLoc)
    
    return df

Now we can simply run this last function on each season and each postseason. With that, our data cleaning is complete.

In [29]:
# ###7.5 seconds to run
# start_time = time.time()

# postseason12 = create_season_df(postseason, 2012)

# end_time = time.time()
# print('Execution time = %.6f seconds' % (end_time-start_time))

Execution time = 658.176695 seconds


In [45]:
# postseason13 = create_season_df(postseason, 2013)
# postseason14 = create_season_df(postseason, 2014)
# postseason15 = create_season_df(postseason, 2015)
# postseason16 = create_season_df(postseason, 2016)
# postseason17 = create_season_df(postseason, 2017)
# postseason18 = create_season_df(postseason, 2018)
# postseason19 = create_season_df(postseason, 2019)

In [46]:
# start_time = time.time()

# regularseason12 = create_season_df(regularseason, 2012)

# end_time = time.time()
# print('Execution time = %.6f seconds' % (end_time-start_time))

Execution time = 627.307349 seconds


In [47]:
# regularseason13 = create_season_df(regularseason, 2013)

In [48]:
# regularseason14 = create_season_df(regularseason, 2014)

In [49]:
# regularseason15 = create_season_df(regularseason, 2015)

In [ ]:
# regularseason16 = create_season_df(regularseason, 2016)

In [ ]:
# regularseason17 = create_season_df(regularseason, 2017)

In [ ]:
# regularseason18 = create_season_df(regularseason, 2018)

In [ ]:
# regularseason19 = create_season_df(regularseason, 2019)

In [ ]:
# regularseason20 = create_season_df(regularseason, 2020)

Because this is so computationally expensive we can save these data frames as csv files so we never need to create these dataframes again. So We'll comment out those lines of code.

In [50]:
# postseason12.to_csv('./data/postseason12.csv')
# postseason13.to_csv('./data/postseason13.csv')
# postseason14.to_csv('./data/postseason14.csv')
# postseason15.to_csv('./data/postseason15.csv')
# postseason16.to_csv('./data/postseason16.csv')
# postseason17.to_csv('./data/postseason17.csv')
# postseason18.to_csv('./data/postseason18.csv')
# postseason19.to_csv('./data/postseason19.csv')

In [51]:
# regularseason12.to_csv('./data/regularseason12.csv')
# regularseason13.to_csv('./data/regularseason13.csv')
# regularseason14.to_csv('./data/regularseason14.csv')
# regularseason15.to_csv('./data/regularseason15.csv')